In [1]:
  # Libraries

import pandas as pd
import pulp 
from pulp import *
import itertools
import numpy as np
from numpy import array

df = pd.read_excel(r'C:\Users\PC\OneDrive - Universidad del Valle de Guatemala\special topics\BPAC\1. Model Parameters\MP16RUN.xlsx')
#print(df)

U=df['U']  #utilization rate
U=U*0.01 

MTBF=df['MTBF']  # mean time between failures
MTTR=df['MTTR']  # mean time between to repair
M1=df['M1']  # service rate of upstream batch process server (batch/hr)
M2=df['M2']  # service rate of downstream single process server (batch/hr)
LT=df['LT']  # PT upper limit for jobs in the downstream queue
B=df['B']  # maximum batch size 
  
    # Derivated variables

f=1/MTBF #failure rate f1=f2=f
r=1/MTTR #repair rate r1=r2=r
n=MTBF/(MTBF+MTTR) # server availability n1=n2=n
Q_qw=100
Q_qe=10

    # Define the cost function

c_hc1 = 0.5  # holding cost rate for WIP at arrival queue
c_hc2 = 0.3  # holding cost rate for WIP at downstream queue
c_sc = 50  # scrap cost
w = 1/LT  # scrap rate for each job in downstream queue

# Uniformization rate
def phi_(M1, M2, y1, y2, r, f, w, Q_qe):
    return M1 + M2 + y1 + y2 + r + 2*f + Q_qe*w

# External job arrival rate
def lambda_(M1, B, n, U, M2):
    return min((M1*B*n*U), (M2*n*U))

def c(y1, y2, sig1, sig2):
    return (c_hc1 * y1 + c_hc2 * y2 + (w * c_sc * y2 if sig2 == 1 else 0)) / phi_

def W(y1, y2, sig1, sig2, V):
    return c(y1, y2, sig1, sig2) + lambda_ / phi_ * V[y1 + 1][y2][sig1][sig2] \
           + (sig1 * M2) / phi_ * V[y1][y2 - 1][sig1][sig2] \
           + (y2 * w) / phi_ * V[y1][y2 - 1][sig1][sig2] \
           + f / phi_ * V[y1][y2][0][sig2] + f / phi_ * V[y1][y2][sig1][0] \
           + r / phi_ * V[y1][y2][sig1][sig2] + r / phi_ * V[y1][y2][sig1][1] \
           + (phi_ * (lambda_ + sig1 * M1 + sig2 * M2 + y2 * w + 2 * r + 2 * f)) / phi_ * V[y1][y2][sig1][sig2]

# Define the state space S
y1_vals = list(range(121))
y2_vals = list(range(21))
sig_vals = [0, 1]
S = [(y1, y2, sig1, sig2) for y1 in y1_vals for y2 in y2_vals for sig1 in sig_vals for sig2 in sig_vals]

# Define the initial value function V[0]
V = [[[[0 for sig2 in range(2)] for sig1 in range(2)] for y2 in range(21)] for y1 in range(121)]


# Define the value iteration algorithm

V = {}
for y1 in range(121):
    for y2 in range(21):
        for sig1 in range(2):
            for sig2 in range(2):
                V[(y1, y2, sig1, sig2)] = 0

for n in range(1000):
    for y1 in range(121):
        for y2 in range(21):
            for sig1 in range(2):
                for sig2 in range(2):
                    W = c(y1, y2, sig1, sig2) + (lambda_/phi_) * V.get((y1+1, y2, sig1, sig2), 0) \
                        + (sig1*M2/phi_) * V.get((y1, y2-1, sig1, sig2), 0) + (y2*w/phi_) * V.get((y1, y2-1, sig1, sig2), 0) \
                        + (f/phi_) * V.get((y1, y2, 0, sig2), 0) + (f/phi_) * V.get((y1, y2, sig1, 0), 0) \
                        + (r/phi_) * V.get((y1, y2, sig1, sig2), 0) + (r/phi_) * V.get((y1, y2, sig1, 1), 0) \
                        + (phi_*(lambda_+sig1*M1+sig2*M2+y2*w+2*r+2*f)/phi_) * V.get((y1, y2, sig1, sig2), 0)
                    a = LpVariable("a", lowBound=0, upBound=1, cat="Integer")
                    prob = LpProblem("MDP", LpMinimize)
                    prob += W == a*(sig1*M1/phi_) * V.get((max(y1-B, 0), min(y2+y1, 20), sig1, sig2), 0) \
                        + (1-a)*(sig1*M1/phi_) * V.get((y1, y2, sig1, sig2), 0) + (sig2*M2/phi_) * V.get((y1, max(y2-1, 0), sig1, sig2), 0)
                    prob.solve()
                    V[(y1, y2, sig1, sig2)] = prob.objective.value()



# Define the objective function
prob += lpSum(c(y1, y2, sig1, sig2) * A[(y1, y2, sig1, sig2)] for (y1, y2, sig1, sig2) in S)

# Define the constraints
for (y1, y2, sig1, sig2) in S:
    if y2 == 0:
        prob += A[(y1, y2, sig1, sig2)] == 0
    else:
        prob += A[(y1, y2, sig1, sig2)] <= 1
        prob += A[(y1, y2, sig1, sig2)] >= 0
        prob += lpSum(A[(y1 - min(y1, B), y2 + min(y1, B), sig1, sig2)] for (y1, y2, sig1, sig2) in S) \
                + lpSum(A[(y1, y2, sig1, sig2)] for (y1, y2, sig1, sig2) in S) == 1
        prob += lpSum(A[(y1, y2 - 1, sig1, sig2)] for (y1, y2, sig1, sig2) in S) == 1

# Solve the LP problem
prob.solve()

# Update the value function
for y1 in range(121):
    for y2 in range(21):
        for sig1 in range(2):
            for sig2 in range(2):
                W = c(y1, y2, sig1, sig2) + lambda_/phi_ * V[y1+1, y2, sig1, sig2] + (sig1*M2)/phi_ * V[y1, y2-1, sig1, sig2] + ((y2*w)/phi_) * V[y1, max(0, y2-1), sig1, sig2] + f/phi_ * V[y1, y2, 0, sig2] + f/phi_ * V[y1, y2, sig1, 0] + r/phi_ * V[y1, y2, sig1, sig2] + r/phi_ * V[y1, y2, sig1, 1] + (phi_*(lambda_+sig1*M1+sig2*M2+y2*w+2*r+2*f))/phi_ * V[y1, y2, sig1, sig2]
                
                a_star = 0
                min_value = float('inf')
                for a in [0, 1]:
                    value = ((a*sig1*M1)/phi_) * V[max(0, y1-B), min(y2+y1, LT), sig1, sig2] + (((1-a)*sig1*M1)/phi_) * V[y1, y2, sig1, sig2] + (sig2*M2/phi_) * V[y1, max(0, y2-1), sig1, sig2] + W
                    if value < min_value:
                        a_star = a
                        min_value = value
                        
                V_new[y1, y2, sig1, sig2] = min_value
                pi[y1, y2, sig1, sig2] = a_star

# Define the optimal value function and decision rules
optimal_value_function = np.zeros((121, 21, 2, 2))
decision_rules = np.zeros((121, 21, 2))

# Extract the optimal value function and decision rules
for y1 in range(121):
    for y2 in range(21):
        for sig1 in range(2):
            for sig2 in range(2):
                min_cost = float('inf')
                best_a = None
                for a in range(2):
                    cost = c(y1, y2, sig1, sig2)
                    if sig1 == 1:
                        cost += (a * M1 / phi) * optimal_value_function[max(y1-B, 0), min(y2+y1, 20), sig1, sig2]
                        cost += ((1 - a) * M1 / phi) * optimal_value_function[y1, y2, sig1, sig2]
                    if sig2 == 1:
                        cost += M2 / phi * optimal_value_function[y1, max(y2-1, 0), sig1, sig2]
                    cost += lambda_ / phi * optimal_value_function[min(y1+1, 120), y2, sig1, sig2]
                    cost += (sig1 * M2 / phi) * optimal_value_function[y1, max(y2-1, 0), sig1, sig2]
                    cost += (y2 * w / phi) * optimal_value_function[y1, max(y2-1, 0), sig1, sig2]
                    cost += f / phi * optimal_value_function[y1, y2, 0, sig2]
                    cost += f / phi * optimal_value_function[y1, y2, sig1, 0]
                    cost += r / phi * optimal_value_function[y1, y2, sig1, sig2]
                    cost += r / phi * optimal_value_function[y1, y2, sig1, 1]
                    cost += (phi_ * (lambda_ + sig1 * M1 + sig2 * M2 + y2 * w + 2 * r + 2 * f)) / phi * optimal_value_function[y1, y2, sig1, sig2]
                    if cost < min_cost:
                        min_cost = cost
                        best_a = a
                optimal_value_function[y1, y2, sig1, sig2] = min_cost
                decision_rules[y1, y2, sig1] = best_a
# Print optimal value function and decision rules
print("\nOptimal Value Function:")
for sig1 in [0, 1]:
    for sig2 in [0, 1]:
        print("sig1 =", sig1, ", sig2 =", sig2)
        for y1 in range(121):
            for y2 in range(21):
                print("V*(", y1, ",", y2, ",", sig1, ",", sig2, ") =", V_star[y1, y2, sig1, sig2])
        print("\n")

print("Decision Rules:")
for sig1 in [0, 1]:
    for sig2 in [0, 1]:
        print("sig1 =", sig1, ", sig2 =", sig2)
        for y1 in range(121):
            for y2 in range(21):
                print("a*(", y1, ",", y2, ",", sig1, ",", sig2, ") =", a_star[y1, y2, sig1, sig2])
        print("\n")




TypeError: unsupported operand type(s) for /: 'float' and 'function'

** Update the value function **
This code loops over all the possible states of the system, computes the value of the W function for each state, and then computes the value of the optimal policy and the new value function using the Bellman optimality equation. The optimal policy is the value of a_star that minimizes the value of the Bellman equation.

** define the optimal value function and decision rules ** In this code, we first define two numpy arrays: optimal_value_function and decision_rules. These arrays have the same dimensions as the V and a variables in the value iteration algorithm. The optimal_value_function array will hold the optimal expected cost-to-go values for each state, while the decision_rules array will hold the optimal decision (0 or 1) for each state.

We then use four nested loops to iterate over all possible states, and for each state, we calculate the minimum cost and corresponding optimal decision by iterating over all possible decisions. We use the c() function to calculate the immediate cost for the current state and decision, and we use the optimal_value_function array to calculate the expected future cost for each possible transition. We then update the optimal_value_function array with the minimum cost, and update the decision_rules array with the optimal decision.

After this loop completes, the optimal_value_function and decision_rules